# Вебинар 1. Введение, примеры задач, бизнес- и ML-метрики 

In [ ]:
import pandas as pd
import numpy as np

# 2. ML-mетрики качества

### ДЗ
реализовать код следующих функций reciprocal_rank, MAP@k, money_recall_at_k, recall_at_k, precision_at_k, money_precision_at_k

*Хороши ли рекомендации, предлагаемые моделью?*  
Обычно считаются для каждого юзера, затем усредняются по юзерам

Представим, что в магазине всего 10 товаров

In [322]:
recommended_list = [1431, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]

### 1. Hit rate

Hit rate = был ли хотя бы 1 релевантный товар среди рекомендованных

- Иногда применяется, когда продаются достаточно дрогие товары (например, бытовая техника) 

----
Hit rate = (был ли хотя бы 1 релевантный товар среди рекомендованных)   

Hit rate@k = (был ли хотя бы 1 релевантный товар среди топ-k рекомендованных)

In [323]:
data = pd.DataFrame({"user_id": ["u1","u2","u3", "u4"], 
              "recommended_list": [
                                      [143, 156, 1134, 991, 27],
                                      [1543, 3345, 533, 11, 43],
                                      [156, 3345, 10, 15, 1134],
                                      [106, 305, 10, 19, 1034]
                                  ],
             "bought_list": [
                             [156,27],
                             [11,43],
                             [1],
                             [10,19]
                            ]})

data

,user_id,recommended_list,bought_list
0,u1,"[143, 156, 1134, 991, 27]","[156, 27]"
1,u2,"[1543, 3345, 533, 11, 43]","[11, 43]"
2,u3,"[156, 3345, 10, 15, 1134]",[1]
3,u4,"[106, 305, 10, 19, 1034]","[10, 19]"


In [324]:
def hit_rate(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    hit_rate = (flags.sum() > 0) * 1
    #flags.sum() > 0 возвращает TRUE 1 или FALSE 0
    return hit_rate


def hit_rate_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    hit_rate = (flags.sum() > 0) * 1

    
    return hit_rate

In [325]:
hit_rate(recommended_list, bought_list)

1

In [326]:
hit_rate_at_k(recommended_list, bought_list, k=5)

1

### hit rate at k усредненое по всем пользователям

In [327]:
def hit_rate_at_k_mean(recommended_list, bought_list, k=5):
    
    len_of_df = recommended_list.shape[0]
    hit_rate_list = []
    for rec, boug in zip(recommended_list,bought_list):
        bought = np.array(boug)
        recommended = np.array(rec)[:k]
        
        flags = np.isin(recommended, bought)
        
        hit_rate = (flags.sum() > 0) * 1
        hit_rate_list.append(hit_rate)
    hit_rate_sum = 0
    for hit in hit_rate_list:   
        hit_rate_sum += hit
            
    hit_rate_mean = hit_rate_sum/ len_of_df

    return hit_rate_mean

In [328]:
def hit_rate_at_k_mean_v2(recommended_list, bought_list, k=5):
    
    hit_rate_np = np.array([])
    for rec, boug in zip(recommended_list,bought_list):
        bought = np.array(boug)
        recommended = np.array(rec)[:k]
        
        flags = np.isin(recommended, bought)
        
        hit_rate = (flags.sum() > 0) * 1
        hit_rate_np = np.append(hit_rate_np, hit_rate)
        
    return hit_rate_np.mean()

In [329]:
hit_rate_at_k_mean_v2(data['recommended_list'], data['bought_list'], k=3)

0.5

In [330]:
hit_rate_at_k_mean(data['recommended_list'], data['bought_list'], k=9)

0.75

### 2. Precision

*Precision* - доля релевантных товаров среди рекомендованных = Какой % рекомендованных товаров  юзер купил

- Пожалуй, самая приближенная к бизнес-метрикам и самая популярная метрика

---

Precision= (# of recommended items that are relevant) / (# of recommended items)  

Precision@k = (# of recommended items @k that are relevant) / (# of recommended items @k)

Money Precision@k = (revenue of recommended items @k that are relevant) / (revenue of recommended items @k)  

**Note:** Обычно k в precision@k достаточно невелико (5-20) и определяется из бизнес-логики. Например, 5 товаров в e-mail рассылке, 20 ответов на первой странице google и т.д

Красная рыба - 400 руб  
Молоко - 60 руб  
Хлеб = 40 руб  
Гречка = 40 руб  
Шоколад = 90 руб  

------  
Варенье - 240 руб  
...  

**Case 1**  
prices_resommended = [400, 60, 40, 40 , 90]  
flags = [1, 0, 0, 0 , 1]  

$precison@5 = \frac{1 + 0 + 0 +0 + 1}{1+1+1+1+1} = 40\%$  
$money precision@5 = \frac{1*400 + 0*60 + ... + 1*90}{1*400 + 1*60 + ... + 1*90} = 77.7\%$  

  
**Case 2**   
prices_resommended = [400, 60, 40, 40 , 90]  
flags = [0, 1, 0, 0 , 1]  

$precison@5 = \frac{0 + 1 + 0 +0 + 1}{1+1+1+1+1} = 40\%$  
$money precision@5 = \frac{0*400 + 1*60 + ... + 1*90}{1*400 + 1*60 + ... + 1*90} = 15.8\%$

In [331]:
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision


def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  
    recommended_list = recommended_list[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision


def money_precision_at_k(recommended_list, bought_list, money_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    money_list = np.array(money_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    money_list = money_list[:k]
    
    flags = np.isin(recommended_list, bought_list)
    flags_money = flags@money_list.T
    precision =flags_money.sum() / len(recommended_list)
        
    
    return precision

In [332]:
money_list = [100,2000,300,40,60,700,900,900,900,800,999]

In [333]:
money_precision_at_k(data['recommended_list'][1], data['bought_list'][1], money_list, k=5)

20.0

In [334]:
data.insert(loc=3, column='money',value = [[20,20,900,80,90],[11,900,66,40,33], [200,200,900,8,190],[91,980,6,40,383]])

In [335]:
data

,user_id,recommended_list,bought_list,money
0,u1,"[143, 156, 1134, 991, 27]","[156, 27]","[20, 20, 900, 80, 90]"
1,u2,"[1543, 3345, 533, 11, 43]","[11, 43]","[11, 900, 66, 40, 33]"
2,u3,"[156, 3345, 10, 15, 1134]",[1],"[200, 200, 900, 8, 190]"
3,u4,"[106, 305, 10, 19, 1034]","[10, 19]","[91, 980, 6, 40, 383]"


### Money precision at k рекомендаций для всех пользователей

In [397]:
def money_precision_at_k_mean(recommended_list, bought_list, money_list, k=5):
    
    np_precision = np.array([]) 
    for rec, boug, mon in zip(recommended_list,bought_list, money_list):
        bought = np.array(boug)
        recommended = np.array(rec)[:k]
        money = np.array(mon)[:k]

        flags = np.isin(recommended, bought)
        flags_money = flags*money
        precision =flags_money.sum() / len(rec)
        np_precision = np.append(np_precision, precision)
    
    return np_precision.mean()

In [398]:
money_precision_at_k_mean(data['recommended_list'], data['bought_list'], data['money'], k=4)

5.3

### 3. Recall

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

- Обычно используется для моделей пре-фильтрации товаров (убрать те товары, которые точно не будем рекомендовать)

---

Recall= (# of recommended items that are relevant) / (# of relevant items)  

Recall@k = (# of recommended items @k that are relevant) / (# of relevant items)

Money Recall@k = (revenue of recommended items @k that are relevant) / (revenue of relevant items)  

    
  
**Note:** в recall@k число k обычно достаточно большое (50-200), больше чем покупок у среднестатистического юзера

In [348]:
def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall


def recall_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    
    return recall


def money_recall_at_k(recommended_list, bought_list, money_list, k=5):
    
    money_list = np.array(money_list)
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    money_list = money_list[:k]
    
    flags = np.isin(recommended_list, bought_list)
    flags_money = money_list*flags
    recall = flags_money.sum() / len(bought_list)
    
    
    return recall

In [349]:
recall_at_k(data['recommended_list'][0], data['bought_list'][0], k=5)

1.0

In [350]:
money_recall_at_k(data['recommended_list'][1], data['bought_list'][1], data['money'][1], k=5)

36.5

In [351]:
money_recall_at_k(data['recommended_list'][0], data['bought_list'][0], data['money'][0])

55.0

### Recall at K рекомендаций для все пользователей

In [395]:
def money_recall_at_k_mean(recommended_list, bought_list, money_list, k=5):
    
    np_recall = np.array([]) 
    for rec, boug, mon in zip(recommended_list,bought_list, money_list):
        bought = np.array(boug)
        recommended = np.array(rec)[:k]
        money = np.array(mon)[:k]
        
        flags = np.isin(recommended, bought)
        flags_money = flags@money
        recall = flags_money.sum() / len(boug)
        np_recall = np.append(np_recall, recall)
    
    return np_recall.mean()

In [396]:
money_recall_at_k_mean(data['recommended_list'], data['bought_list'], data['money'], k=3)

3.25

# Метрики ранжирования

Если важен порядок рекомендаций. Подробнее можно почитать [здесь](https://habr.com/ru/company/econtenta/blog/303458/). Формулы в статье могут несколько отличаться от формул в лекции 

## AP@k
AP@k - average precision at k

$$AP@k = \frac{1}{r} \sum{[recommended_{relevant_i}] * precision@k}$$

- r - кол-во релевантный среди рекомендованных
- Суммируем по всем релевантным товарам
- Зависит от порядка реокмендаций

In [418]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(k):
        
        if flags[i]:
            p_k = precision_at_k(recommended_list, bought_list, k=i+1)
            sum_ += p_k
            
    result = sum_ / k
    
    return result

In [419]:
ap_k(data['recommended_list'][0], data['bought_list'][0], k=3)

0.16666666666666666

### MAP@k

MAP@k (Mean Average Precision@k)  
Среднее AP@k по всем юзерам
- Показывает средневзвешенную точность рекомендаций

$$MAP@k = \frac{1}{|U|} \sum_u{AP_k}$$
  
|U| - кол-во юзеров

### MAP at K рекомендаций по всем пользователем 

In [415]:
def map_k_mean(recommended_list, bought_list, k=5):
    np_map = np.array([]) 
    for rec, boug in zip(recommended_list, bought_list):
        bought_list = np.array(boug)
        recommended_list = np.array(rec)
    
        flags = np.isin(recommended_list, bought_list)
    
        if sum(flags) == 0:
            result = 0
    
        sum_ = 0
        for i in range(k):
        
            if flags[i]:
                p_k = precision_at_k(recommended_list, bought_list, k=i+1)
                sum_ += p_k
            
        result = sum_ / k
        np_map = np.append(np_map, result)
        
    return np_map.mean()
    

    

In [416]:
map_k_mean(data['recommended_list'], data['bought_list'], k=5)

0.11916666666666667

### AUC@k
AUC для первых k наблюдений  
- Можно посчитать как sklern.metrics.roc_auc_score для топ-k предсказаний
- Показывает долю верно отранжированных товаров

### NDCG@k
Normalized discounted cumulative gain

$$DCG = \frac{1}{|r|} \sum_u{\frac{[bought fact]}{discount(i)}}$$  

$discount(i) = 1$ if $i <= 2$,   
$discount(i) = log_2(i)$ if $i > 2$


(!) Считаем для первых k рекоммендаций   
(!) - существуют вариации с другими $discount(i)$  
i - ранк рекомендованного товара  
|r| - кол-во рекомендованных товаров 

$$NDCG = \frac{DCG}{ideal DCG}$$


In [ ]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]

$DCG@5 = \frac{1}{5}*(1 / 1 + 0 / 2 + 0 / log(3) + 1 / log(4) + 0 / log(5))$  
$ideal DCG@5 = \frac{1}{5}*(1 / 1 + 1 / 2 + 1 / log(3) + 1 / log(4) + 1 / log(5))$  

$NDCG = \frac{DCG}{ideal DCG}$

### MRR@k
Mean Reciprocal Rank

- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $k_u$
- Посчитать reciprocal rank = $\frac{1}{k_u}$

$$MRR = mean(\frac{1}{k_u})$$

In [477]:
def reciprocal_rank(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    flags = np.isin(recommended_list, bought_list)
    
    if (flags.sum() > 0):
        np_mrr = np.array([])
        for i, val in enumerate(flags):
            if val:
                mrr = 1/(i+1)
                np_mrr = np.append(np_mrr, mrr)
                break
        return np_mrr[0]
    else:
        return 0
        

reciprocal_rank(data['recommended_list'][2], data['bought_list'][2], k=5)

0

In [478]:
reciprocal_rank(data['recommended_list'][0], data['bought_list'][0], k=5)

0.5

### reciprocal_rank усредненый по всем пользователям

In [480]:
def reciprocal_rank_mean(recommended_list, bought_list, k=5):
    np_r = np.array([]) 
    for rec, boug in zip(recommended_list,bought_list):
        bought_list = np.array(boug)
        recommended_list = np.array(rec)[:k]
        flags = np.isin(recommended_list, bought_list)
        if (flags.sum() > 0):
            for i, val in enumerate(flags):
                if val:
                    mrr = 1/(i+1)
                    np_r = np.append(np_r, mrr)
                    break
        else:
            np_r = np.append(np_r,0)
    return np_r.mean()

        
        


In [481]:
reciprocal_rank_mean(data['recommended_list'], data['bought_list'], k=5)

0.2708333333333333

### NDSG

In [386]:
def NDCG(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    flags = np.isin(bought_list, recommended_list)
    
    
    DCG_at_k = sum(np.array([el/np.log(i+2) for i, el in enumerate(flags)])/k)
    idealDCG_at_k = sum(np.array([el/np.log(i+2) for i, el in enumerate(np.ones(k))])/k)
    
    
    NDSG = DCG_at_k/idealDCG_at_k
  

    return NDSG

In [387]:
NDCG(recommended_list, bought_list)

0.14606834984270647

### NDCG at K рекомендаций усредненый по всем пользователям

In [484]:
def NDCG_mean(recommended_list, bought_list, k=5):
    ndcg = np.array([]) 
    for rec, boug in zip(recommended_list,bought_list):
        bought_list = np.array(boug)
        recommended_list = np.array(rec)[:k]
        flags = np.isin(recommended_list, bought_list)
        DCG_at_k = sum(np.array([el/np.log(i+2) for i, el in enumerate(flags)])/k)
        idealDCG_at_k = sum(np.array([el/np.log(i+2) for i, el in enumerate(np.ones(k))])/k)
        NDSG = DCG_at_k/idealDCG_at_k
        ndcg = np.append(ndcg, NDSG)
  

    return ndcg.mean()

In [485]:
NDCG_mean(data['recommended_list'], data['bought_list'], k=5)

0.23452830552035803